<a href="https://colab.research.google.com/github/bhavanakodali5/ObjectDectection-IIITH/blob/main/African_Wildlife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- Google Colab Setup ---
# This section prepares your Colab environment.

# 1. Mount Google Drive
# This allows you to save your trained models and results persistently,
# so you don't lose them when your Colab session ends.
# You'll be prompted to authorize Google Drive access.
from google.colab import drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted at /content/drive")

# Define the path where you want to save your results within Google Drive
# It's good practice to create a specific folder for your project.
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/YOLOv8_African_Wildlife_Results'
# Create the directory in Google Drive if it doesn't exist
import os
os.makedirs(DRIVE_PROJECT_PATH, exist_ok=True)
print(f"Results will be saved to: {DRIVE_PROJECT_PATH}")


# 2. Install Ultralytics YOLOv8
# This command installs the necessary library for YOLOv8.
# The `-q` flag keeps the output quiet.
print("\nInstalling Ultralytics YOLOv8...")
!pip install -q ultralytics
print("Ultralytics YOLOv8 installed.")

# 3. Change directory to /content
# This is a common and convenient location in Colab for temporary files and datasets.
%cd /content
print("\nChanged current directory to /content")

# 4. Download the African Wildlife Dataset YAML file
# This YAML file contains the configuration for the African Wildlife dataset,
# including instructions for Ultralytics to automatically download the actual image data.
print("Downloading dataset configuration file 'african-wildlife.yaml'...")
!wget -q https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/african-wildlife.yaml
print("'african-wildlife.yaml' downloaded.")

# --- YOLOv8 Training Script ---
# This is the core script for training your object detection model.

# Import the YOLO class from the ultralytics library
from ultralytics import YOLO
import os
import shutil # For copying files

# --- Configuration ---
# Set the desired number of training epochs.
# The user requested anywhere between 10 to 25 epochs. We'll use 20 for this example.
NUM_EPOCHS = 20

# Define the path to your dataset's YAML file.
# We've already downloaded 'african-wildlife.yaml' to /content.
DATASET_YAML_PATH = 'african-wildlife.yaml'

# Define the model to use. 'yolov8n.pt' is the smallest and fastest model.
# You can try 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', or 'yolov8x.pt' for better accuracy
# but slower training/inference. Ultralytics will download this model automatically if not found.
MODEL_NAME = 'yolov8n.pt'

# Define the image size for training. 640 is a common default for YOLOv8.
IMAGE_SIZE = 640

# --- Model Training Function ---
def train_yolov8_model():
    """
    Loads a YOLOv8 model, trains it on the African Wildlife dataset,
    and saves the training results.
    """
    print(f"\n--- Starting YOLOv8 training for {MODEL_NAME} ---")
    print(f"Dataset YAML: {DATASET_YAML_PATH}")
    print(f"Epochs: {NUM_EPOCHS}")
    print(f"Image Size: {IMAGE_SIZE}")

    # Verify that the dataset YAML file exists before proceeding.
    if not os.path.exists(DATASET_YAML_PATH):
        print(f"Error: Dataset YAML file not found at '{DATASET_YAML_PATH}'")
        print("Please ensure it was downloaded correctly in the previous steps.")
        print("Refer to: https://docs.ultralytics.com/datasets/detect/african-wildlife/#dataset-yaml")
        return

    # Inform the user where the dataset is expected to be downloaded by Ultralytics
    expected_dataset_path = "/content/datasets/african-wildlife"
    print(f"The African Wildlife dataset will be automatically downloaded and extracted to: {expected_dataset_path}")

    try:
        # Load a pre-trained YOLOv8 model.
        # This will automatically download the weights if they are not already present locally.
        model = YOLO(MODEL_NAME)
        print(f"Model '{MODEL_NAME}' loaded successfully.")

        # Train the model.
        # Ultralytics will save results to a 'runs' directory in the current working directory,
        # which is still /content at this point.
        results = model.train(data=DATASET_YAML_PATH, epochs=NUM_EPOCHS, imgsz=IMAGE_SIZE)

        print("\n--- Training complete! ---")
        # Identify the training run's output directory
        local_runs_dir = '/content/runs/detect'
        # Get the latest run folder (e.g., 'train', 'train1', 'train2')
        latest_run_folder = sorted([d for d in os.listdir(local_runs_dir) if os.path.isdir(os.path.join(local_runs_dir, d))])[-1]
        source_results_path = os.path.join(local_runs_dir, latest_run_folder)

        print(f"Results initially saved locally to: {source_results_path}")

        # Copy results to Google Drive
        destination_results_path = os.path.join(DRIVE_PROJECT_PATH, latest_run_folder)
        print(f"Copying results to Google Drive: {destination_results_path}...")
        shutil.copytree(source_results_path, destination_results_path)
        print("Results successfully copied to Google Drive!")

        print(f"You can find your results (graphs, best.pt model, etc.) here: {destination_results_path}")


    except Exception as e:
        print(f"\nAn error occurred during training or saving results: {e}")
        print("Please review the error message and ensure your Colab environment,")
        print("installations, and dataset configuration are correct. Also check Google Drive permissions.")
    finally:
        # Always try to list the contents of the datasets directory
        print(f"\n--- Verifying dataset directory contents ({expected_dataset_path}) ---")
        if os.path.exists(expected_dataset_path):
            !ls -l {expected_dataset_path}
        else:
            print(f"The dataset directory {expected_dataset_path} does not exist.")
            print("This indicates the dataset was likely not downloaded or extracted successfully during training.")
            print("Check the training output above for any download errors.")


# --- Execute Training ---
# This ensures the training function runs when the script is executed.
if __name__ == '__main__':
    train_yolov8_model()

Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted at /content/drive
Results will be saved to: /content/drive/MyDrive/YOLOv8_African_Wildlife_Results

Installing Ultralytics YOLOv8...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00


100%|██████████| 6.25M/6.25M [00:00<00:00, 245MB/s]

Model 'yolov8n.pt' loaded successfully.
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=african-wildlife.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=True

100%|██████████| 100M/100M [00:00<00:00, 391MB/s] 
Unzipping /content/datasets/african-wildlife.zip to /content/datasets/african-wildlife...: 100%|██████████| 3008/3008 [00:00<00:00, 3186.83file/s]

...

      20/20         0G     0.5229      0.493      0.981         20        640: 100%|██████████| 66/66 [15:21<00:00, 13.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:52<00:00,  6.51s/it]

                   all        225        379      0.941      0.878      0.946      0.785



20 epochs completed in 5.355 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:47<00:00,  5.91s/it]


                   all        225        379      0.941      0.878      0.946      0.785
               buffalo         62         89       0.97      0.865      0.952      0.808
              elephant         53         91      0.871      0.857      0.922      0.761
                 rhino         55         85      0.963      0.923      0.957      0.828
                 zebra         59        114      0.959      0.868      0.955      0.742
Speed: 2.1ms preprocess, 196.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train

--- Training complete! ---
Results initially saved locally to: /content/runs/detect/train
Copying results to Google Drive: /content/drive/MyDrive/YOLOv8_African_Wildlife_Results/train...
Results successfully copied to Google Drive!
You can find your results (graphs, best.pt model, etc.) here: /content/drive/MyDrive/YOLOv8_African_Wildlife_Results/train

--- Verifying dataset directory contents (/content/datasets/african-wildlife) -